In [1]:
# импортируем необходимые библиотеки, классы и функции
import pandas as pd
import numpy as np
import pathlib
from tqdm.notebook import tqdm

from etna.datasets import TSDataset
from etna.pipeline import Pipeline
from etna.metrics import MSE
from etna.transforms import (LagTransform,
                             MinMaxScalerTransform,
                             MeanTransform,
                             DateFlagsTransform)

from etna_utils import LGBMMultiSegmentModel

# отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

/Users/artemgruzdev/anaconda3/lib/python3.10/site-packages/etna/settings.py:61: UserWarning: `tsfresh` is not available, to install it, run `pip install tsfresh==0.19.0 && pip install protobuf==3.20.1`
  warnings.warn(


# Загрузка данных

In [2]:
# задаем константную часть пути
DATA_PREFIX = pathlib.PurePath('/Users/artemgruzdev/Documents/GitHub/'
                               'Time_Series/Code/Data/store_sales')
# задаем горизонт
HORIZON = 16
# задаем стартовую дату
start_date = '2015-01-01'

In [3]:
# загружаем исторический набор
train = pd.read_csv(
    DATA_PREFIX.joinpath('train.csv'),
    parse_dates=['date'], 
    infer_datetime_format=True,
)
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
# загружаем набор новых данных
test = pd.read_csv(
    DATA_PREFIX.joinpath('test.csv'), 
    parse_dates=['date'], 
    infer_datetime_format=True,
)
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


# Создание объекта `TSDataset`

In [5]:
# функция подготовки исторического набора эндогенных
# переменных и набора с экзогенными переменными
# (исторический период + прогнозируемый период)
def prepare_df(df, start_date=None, num_segments=None):
    df = df.drop(columns=['onpromotion', 'id'])
    # создаем сегменты
    df['segment'] = [f'{x}::{y}' for x, y in zip(df['store_nbr'], 
                                                 df['family'])]
    # переименовываем столбцы в соответствии с требованиями ETNA
    df.rename(columns={
        'date': 'timestamp', 
        'sales': 'target', 
    }, inplace=True)
    
    # если стартовая дата не задана, используем ранее
    # созданную стартовую дату
    if start_date is not None:
        df = df[df['timestamp'] >= start_date]
    
    # если количество сегментов задано, формируем сегменты на основе 
    # первых num_segments уникальных значений столбца segments
    if num_segments is not None:
        segments = df['segment'].unique()[:num_segments]
        df = df[df['segment'].isin(segments)]
    
    # формируем датафрейм из 3 обязательных столбцов
    df = df[['timestamp', 'segment', 'target']]
    # выполняем логарифмирование зависимой переменной
    df['target'] = np.log1p(df['target'])  
    # стартовая дата
    start_date = df['timestamp'].min()
    # последняя дата: последняя дата исторического набора + горизонт
    end_date = df['timestamp'].max() + pd.Timedelta(HORIZON, 'D')
    # создаем индекс дат
    timestamp = pd.date_range(start=start_date, end=end_date, freq='D')
    # создаем пустой список
    df_exog_list = []
    # создаем датафрейм со столбцом 
    # timestamp на основе индекса дат
    df_exog_segment_template = pd.DataFrame({'timestamp': timestamp})
    # формируем датафрейм с экзогенными переменными
    for segment in tqdm(df['segment'].unique()):
        df_exog_segment = df_exog_segment_template.copy()
        df_exog_segment['segment'] = segment
        df_exog_segment['regressor_store_nbr'] = segment.split('::')[0]
        df_exog_segment['regressor_family'] = segment.split('::')[1]
        df_exog_list.append(df_exog_segment)
    df_exog = pd.concat(df_exog_list, ignore_index=True)
    # признакам, которые хотим обрабатывать как
    # категориальные, присваиваем тип category
    df_exog['regressor_store_nbr'] = df_exog['regressor_store_nbr'].astype(
        'category')
    df_exog['regressor_family'] = df_exog['regressor_family'].astype(
        'category')
    
    return df, df_exog

In [6]:
# создаем исторический набор с эндогенными переменными
# и набор с экзогенными переменными (исторический 
# период + прогнозируемый период)
train, train_exog = prepare_df(train, start_date=start_date)

  0%|          | 0/1782 [00:00<?, ?it/s]

In [7]:
# смотрим набор c эндогенными переменными
train

,timestamp,segment,target
1297296,2015-01-01,1::AUTOMOTIVE,0.000000
1297297,2015-01-01,1::BABY CARE,0.000000
1297298,2015-01-01,1::BEAUTY,0.000000
1297299,2015-01-01,1::BEVERAGES,0.000000
1297300,2015-01-01,1::BOOKS,0.000000
...,...,...,...
3000883,2017-08-15,9::POULTRY,6.084802
3000884,2017-08-15,9::PREPARED FOODS,5.046987
3000885,2017-08-15,9::PRODUCE,7.791824
3000886,2017-08-15,9::SCHOOL AND OFFICE SUPPLIES,4.804021


In [8]:
# смотрим набор с экзогенными переменными
train_exog

,timestamp,segment,regressor_store_nbr,regressor_family
0,2015-01-01,1::AUTOMOTIVE,1,AUTOMOTIVE
1,2015-01-02,1::AUTOMOTIVE,1,AUTOMOTIVE
2,2015-01-03,1::AUTOMOTIVE,1,AUTOMOTIVE
3,2015-01-04,1::AUTOMOTIVE,1,AUTOMOTIVE
4,2015-01-05,1::AUTOMOTIVE,1,AUTOMOTIVE
...,...,...,...,...
1735663,2017-08-27,9::SEAFOOD,9,SEAFOOD
1735664,2017-08-28,9::SEAFOOD,9,SEAFOOD
1735665,2017-08-29,9::SEAFOOD,9,SEAFOOD
1735666,2017-08-30,9::SEAFOOD,9,SEAFOOD


In [9]:
# создаем объединенный набор
ts = TSDataset(
    df=TSDataset.to_dataset(train), 
    df_exog=TSDataset.to_dataset(train_exog), 
    freq='D', known_future='all'
)

In [10]:
# смотрим объединенный набор
ts

segment      10::AUTOMOTIVE                                  10::BABY CARE  \
feature    regressor_family regressor_store_nbr    target regressor_family   
timestamp                                                                    
2015-01-01       AUTOMOTIVE                  10  0.000000        BABY CARE   
2015-01-02       AUTOMOTIVE                  10  1.386294        BABY CARE   
2015-01-03       AUTOMOTIVE                  10  1.609438        BABY CARE   
2015-01-04       AUTOMOTIVE                  10  0.693147        BABY CARE   
2015-01-05       AUTOMOTIVE                  10  1.945910        BABY CARE   
...                     ...                 ...       ...              ...   
2017-08-11       AUTOMOTIVE                  10  1.098612        BABY CARE   
2017-08-12       AUTOMOTIVE                  10  1.098612        BABY CARE   
2017-08-13       AUTOMOTIVE                  10  0.000000        BABY CARE   
2017-08-14       AUTOMOTIVE                  10  0.693147        BABY CARE   
2017-08-15       AUTOMOTIVE                  10  1.609438        BABY CARE   

segment                                     10::BEAUTY                      \
feature    regressor_store_nbr target regressor_family regressor_store_nbr   
timestamp                                                                    
2015-01-01                  10    0.0           BEAUTY                  10   
2015-01-02                  10    0.0           BEAUTY                  10   
2015-01-03                  10    0.0           BEAUTY                  10   
2015-01-04                  10    0.0           BEAUTY                  10   
2015-01-05                  10    0.0           BEAUTY                  10   
...                        ...    ...              ...                 ...   
2017-08-11                  10    0.0           BEAUTY                  10   
2017-08-12                  10    0.0           BEAUTY                  10   
2017-08-13                  10    0.0           BEAUTY                  10   
2017-08-14                  10    0.0           BEAUTY                  10   
2017-08-15                  10    0.0           BEAUTY                  10   

segment                 10::BEVERAGES  ... 9::PREPARED FOODS       9::PRODUCE  \
feature       target regressor_family  ...            target regressor_family   
timestamp                              ...                                      
2015-01-01  0.000000        BEVERAGES  ...          0.000000          PRODUCE   
2015-01-02  1.098612        BEVERAGES  ...          4.821571          PRODUCE   
2015-01-03  0.693147        BEVERAGES  ...          4.110874          PRODUCE   
2015-01-04  1.791759        BEVERAGES  ...          3.920209          PRODUCE   
2015-01-05  1.098612        BEVERAGES  ...          3.608916          PRODUCE   
...              ...              ...  ...               ...              ...   
2017-08-11  1.098612        BEVERAGES  ...          4.728272          PRODUCE   
2017-08-12  1.386294        BEVERAGES  ...          4.874464          PRODUCE   
2017-08-13  1.791759        BEVERAGES  ...          4.665032          PRODUCE   
2017-08-14  1.945910        BEVERAGES  ...          4.745975          PRODUCE   
2017-08-15  1.609438        BEVERAGES  ...          5.046987          PRODUCE   

segment                                  9::SCHOOL AND OFFICE SUPPLIES  \
feature    regressor_store_nbr    target              regressor_family   
timestamp                                                                
2015-01-01                   9  0.000000    SCHOOL AND OFFICE SUPPLIES   
2015-01-02                   9  0.000000    SCHOOL AND OFFICE SUPPLIES   
2015-01-03                   9  0.000000    SCHOOL AND OFFICE SUPPLIES   
2015-01-04                   9  0.000000    SCHOOL AND OFFICE SUPPLIES   
2015-01-05                   9  0.000000    SCHOOL AND OFFICE SUPPLIES   
...                        ...       ...                           ...   
2017-08-11   

In [11]:
# смотрим регрессоры
ts.regressors

['regressor_family', 'regressor_store_nbr']

In [12]:
num_lags = 50

# задаем список преобразований/признаков
transforms = [
    MinMaxScalerTransform(in_column='target'),
    LagTransform(in_column='target', 
                 lags=[HORIZON + i for i in range(num_lags)],
                 out_column='lag'),
    DateFlagsTransform(
        day_number_in_week=True,
        day_number_in_month=True,
        is_weekend=True,
        out_column='datetime'),
    MeanTransform(in_column='target', window=32, out_column='mean32')
]

# Перекрестная проверка расширяющимся окном

In [13]:
# создаем модель
model = LGBMMultiSegmentModel(n_estimators=400,
                              learning_rate=0.08,
                              min_data_in_leaf=80,
                              subsample=0.6)
# передаем в конвейер модель, горизонт 
# и набор преобразований/признаков
pipeline = Pipeline(model=model,
                    horizon=HORIZON, 
                    transforms=transforms)
# запускаем перекрестную проверку 
# расширяющимся окном
metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts, metrics=[MSE()], n_folds=3
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   22.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   46.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.1m

In [14]:
# смотрим метрики по 3 тестовым выборках 
# для первых 2 сегментов
n_folds = 3
n_segments = 2
n = n_folds * n_segments
metrics_df.head(n)

,segment,MSE,fold_number
0,10::AUTOMOTIVE,0.335921,0
0,10::AUTOMOTIVE,0.469482,1
0,10::AUTOMOTIVE,0.404746,2
1,10::BABY CARE,0.000044,0
1,10::BABY CARE,0.000014,1
1,10::BABY CARE,0.000096,2


In [15]:
# смотрим значение RMSE, усредненное по сегментам
metric = metrics_df['MSE'].mean()**(1/2)
print(f'RMSE: {metric:.4f}')

RMSE: 0.4309


# Обучение на всем историческом наборе

In [16]:
# создаем модель
model = LGBMMultiSegmentModel(n_estimators=400,
                              learning_rate=0.08, 
                              min_data_in_leaf=80,
                              subsample=0.6)
# передаем в конвейер модель, горизонт 
# и набор преобразований/признаков
pipeline = Pipeline(model=model, 
                    horizon=HORIZON, 
                    transforms=transforms)

In [17]:
# обучаем конвейер на всем историческом наборе
pipeline.fit(ts=ts);

[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80


In [18]:
# получаем набор с прогнозами
forecasted = pipeline.forecast()
forecasted

segment                  10::AUTOMOTIVE                              \
feature    datetime_day_number_in_month datetime_day_number_in_week   
timestamp                                                             
2017-08-16                           16                           2   
2017-08-17                           17                           3   
2017-08-18                           18                           4   
2017-08-19                           19                           5   
2017-08-20                           20                           6   
2017-08-21                           21                           0   
2017-08-22                           22                           1   
2017-08-23                           23                           2   
2017-08-24                           24                           3   
2017-08-25                           25                           4   
2017-08-26                           26                           5   
2017-08-27                           27                           6   
2017-08-28                           28                           0   
2017-08-29                           29                           1   
2017-08-30                           30                           2   
2017-08-31                           31                           3   

segment                                                                 \
feature    datetime_is_weekend    lag_16    lag_17    lag_18    lag_19   
timestamp                                                                
2017-08-16               False  0.355418  0.000000  0.000000  0.224244   
2017-08-17               False  0.224244  0.355418  0.000000  0.000000   
2017-08-18               False  0.000000  0.224244  0.355418  0.000000   
2017-08-19                True  0.355418  0.000000  0.224244  0.355418   
2017-08-20                True  0.520678  0.355418  0.000000  0.224244   
2017-08-21               False  0.355418  0.520678  0.355418  0.000000   
2017-08-22               False  0.355418  0.355418  0.520678  0.355418   
2017-08-23               False  0.448488  0.355418  0.355418  0.520678   
2017-08-24               False  0.000000  0.448488  0.355418  0.355418   
2017-08-25               False  0.448488  0.000000  0.448488  0.355418   
2017-08-26                True  0.000000  0.448488  0.000000  0.448488   
2017-08-27                True  0.355418  0.000000  0.448488  0.000000   
2017-08-28               False  0.355418  0.355418  0.000000  0.448488   
2017-08-29               False  0.000000  0.355418  0.355418  0.000000   
2017-08-30               False  0.224244  0.000000  0.355418  0.355418   
2017-08-31               False  0.520678  0.224244  0.000000  0.355418   

segment                                   ... 9::SEAFOOD                      \
feature       lag_20    lag_21    lag_22  ...     lag_60    lag_61    lag_62   
timestamp                                 ...                                  
2017-08-16  0.000000  0.224244  0.000000  ...   0.787679  0.719993  0.704803   
2017-08-17  0.224244  0.000000  0.224244  ...   0.810536  0.787679  0.719993   
2017-08-18  0.000000  0.224244  0.000000  ...   0.676614  0.810536  0.787679   
2017-08-19  0.000000  0.000000  0.224244  ...   0.656358  0.676614  0.810536   
2017-08-20  0.355418  0.000000  0.000000  ...   0.697126  0.656358  0.676614   
2017-08-21  0.224244  0.355418  0.000000  ...   0.661814  0.697126  0.656358   
2017-08-22  0.000000  0.224244  0.355418  ...   0.537138  0.661814  0.697126   
2017-08-23  0.355418  0.000000  0.224244  ...   0.801038  0.537138  0.661814   
2017-08-24  0.520678  0.355418  0.000000  ...   0.784231  0.801038  0.537138   
2017-08-25  0.355418  0.520678  0.355418  ...   0.625484  0.784231  0.801038   
2017-08-26  0.355418  0.355418  0.520678  ...   0.597001  0.625484  0.784231   
2017-08-27  0.448488  0.355418  0.355418  ...   0.621126  0.597001  0.625484   
2017-08-28  0.000000  0.448488  0.355418  .

# Формирование файла посылки

In [19]:
# загружаем файл посылки
sample_submission = pd.read_csv(
    DATA_PREFIX.joinpath('sample_submission.csv')
)
# выводим первые 5 наблюдений посылки
sample_submission.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


In [20]:
# выводим первые 5 наблюдений набора новых данных
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [21]:
# берем прогнозы
forecasted_target = forecasted[:, :, 'target']
# создаем серию из столбца sales файла посылки
sales = sample_submission['sales'].copy()
# записываем прогнозы в только что созданную серию
for i, row in tqdm(test.iterrows(), total=test.shape[0]):
    date = row['date']
    segment = f"{row['store_nbr']}::{row['family']}"
    try:
        target = forecasted_target.loc[date, segment].item()
        sales.iloc[i] = target
    except IndexError:
        pass

  0%|          | 0/28512 [00:00<?, ?it/s]

In [22]:
# смотрим статистики серии, выясняем,
# есть ли отрицательные прогнозы
sales.describe()

count    28512.000000
mean         3.619754
std          2.492622
min         -0.040792
25%          1.538671
50%          3.386431
75%          5.578472
max          9.723729
Name: sales, dtype: float64

In [23]:
# серия с прогнозами становится столбцом sales файла посылки
# если есть отрицательные продажи, берем 0, и выполняем 
# экспоненцирование (поскольку ранее логарифмировали
# зависимую переменную)
sample_submission['sales'] = np.expm1(np.maximum(sales, 0))

In [24]:
# смотрим первые 5 наблюдений файла посылки
sample_submission.head()

,id,sales
0,3000888,3.948688
1,3000889,0.007215
2,3000890,3.289083
3,3000891,2385.625169
4,3000892,0.126470


In [25]:
# записываем посылку в виде CSV-файла
sample_submission.to_csv('etna_store_sales_regressors.csv', 
                         index=False)